## Setup

### Imports

In [117]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

### Importação do CSV

In [118]:
csv = pd.read_csv('./resources/superstore.csv')

### Definição de métodos

In [119]:
def filter_columns(df, columns):
    return df.drop(df.columns.difference(columns), axis=1)

## Tratamento do Dataframe

### Padronizar nome das colunas

In [120]:
csv.columns = csv.columns.str.lower()
csv.columns = csv.columns.str.replace(' ', '_')
csv.columns = csv.columns.str.replace('-', '_')

### Remover colunas que não serão utilizadas

In [121]:
columns_to_drop =  [
    'unnamed:_0'
]

csv.drop(columns=columns_to_drop, inplace=True)

In [122]:
csv.columns

Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id',
       'customer_name', 'segment', 'city', 'state', 'country', 'region',
       'market', 'product_id', 'category', 'sub_category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'delivery_days', 'order_year', 'order_month'],
      dtype='object')

### Obter dataframe dos países

In [123]:
import plotly.express as px

gapminder_df = px.data.gapminder().query("year==2007")
gapminder_df = filter_columns(gapminder_df, ['country', 'continent', 'iso_alpha', 'iso_num', 'gdpPercap'])

### Adicionar países que não estão inclusos

In [124]:
new_countries = pd.DataFrame({
    'country': ['Russia', 'Ukraine', 'Kazakhstan', 'South Korea', 'Suriname'],
    'continent': ['Europe', 'Europe', 'Europe', 'Asia', 'Americas'],
    'iso_alpha': ['RUS', 'UKR', 'KAZ', 'KOR', 'SUR'],
    'iso_num': [643, 804, 398, 410, 740]
})

gapminder_df = pd.concat([gapminder_df, new_countries], ignore_index=True)

### Atualizar valores

In [125]:
countries = [
    ('YEM', 'Yemen')
]

for iso_alpha, country in countries:
    gapminder_df.loc[gapminder_df['iso_alpha'] == iso_alpha, 'country'] = country

### Realizando o merge

In [126]:
database = pd.merge(csv, gapminder_df, on="country", how="left")

In [127]:
database.columns

Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id',
       'customer_name', 'segment', 'city', 'state', 'country', 'region',
       'market', 'product_id', 'category', 'sub_category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'delivery_days', 'order_year', 'order_month',
       'continent', 'gdpPercap', 'iso_alpha', 'iso_num'],
      dtype='object')

## Análises

### Quantidade de Vendas por país

In [128]:
sales_per_country = filter_columns(database, ['country', 'iso_alpha'])
sales_per_country.groupby(by=['country', 'iso_alpha'])['country'].count()
sales_per_country = pd.DataFrame({
    'sales' : sales_per_country.groupby(by=['country', 'iso_alpha'])['country'].count()
}).reset_index()

fig = go.Figure(data=go.Choropleth(
    locations=sales_per_country['iso_alpha'],
    z=sales_per_country['sales'],
    text=sales_per_country['country'],
    colorscale='Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Número de vendas',
))

fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    title_text='Quantidade de Vendas por País',
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='equirectangular'
    )
)

fig.show()

### Lucro por país

In [129]:
profit_per_country = filter_columns(database, ['country', 'iso_alpha', 'profit'])

profit_per_country = pd.DataFrame({
    'profit' : profit_per_country.groupby(by=['country', 'iso_alpha'])['profit'].sum()
}).reset_index()

In [130]:
fig = go.Figure(data=go.Choropleth(
    locations = profit_per_country['iso_alpha'],
    z = profit_per_country['profit'],
    text = profit_per_country['country'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'LUCRO',
))

fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    title_text='Lucro por país',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

### Média do tempo de entrega por tipo de entrega em dias

In [131]:
ship_mode_mean = filter_columns(database, ['ship_mode', 'delivery_days'])

ship_mode_mean = pd.DataFrame({
    'mean' : ship_mode_mean.groupby(by=['ship_mode'])['delivery_days'].mean()
}).reset_index()

ship_mode_mean = ship_mode_mean.sort_values(by=['mean'])

In [132]:
long_df = px.data.medals_long()

fig = px.bar(
    ship_mode_mean, 
    x="ship_mode", 
    y="mean", 
    color="ship_mode", 
    title="Média do tempo de entrega por tipo de entrega em dias", 
    pattern_shape="ship_mode"
)

fig.show()

### Utilizando medidas de tendencia central nas vendas

In [133]:
sales_mean = filter_columns(database, ['sales'])

sales_mean = sales_mean.mean()
sales_mean

sales    245.974303
dtype: float64

In [134]:
sales_median = filter_columns(database, ['sales'])

sales_median = sales_median.median()
sales_median

sales    85.0
dtype: float64

In [187]:
sales_bloxplot = filter_columns(database, ['delivery_days'])

pd.options.plotting.backend = "plotly"

fig = sales_bloxplot.boxplot()
fig.show()

In [195]:
sales_deviation = filter_columns(database, ['quantity'])

# sales_deviation = sales_deviation.std()
# sales_deviation

import plotly.express as px

fig = px.histogram(sales_deviation, x="quantity")
fig.show()

### Pedidos por ano

In [137]:
order_per_year = filter_columns(database, ['order_year'])

order_per_year = pd.DataFrame({
    'count' : order_per_year.groupby(by=['order_year'])['order_year'].count()
}).reset_index()

order_per_year = order_per_year.sort_values(by=['count'])
order_per_year['order_year'] = order_per_year['order_year'].astype(str)
order_per_year

,order_year,count
0,2012,8998
1,2013,10962
2,2014,13799
3,2015,17531


In [138]:
fig = px.bar(
    order_per_year,
    x='order_year', 
    y='count', 
    color='order_year', 
    pattern_shape='order_year'
)

fig.show()

### Pedidos por mês e ano

In [139]:
order_per_month = filter_columns(database, ['order_year', 'order_month'])

order_per_month = pd.DataFrame({
    'count' : order_per_month.groupby(by=['order_month', 'order_year'])['order_month'].count()
}).reset_index()

order_per_month = order_per_month.sort_values(by=['order_year', 'order_month'])
order_per_month['date'] = order_per_month.apply(lambda order: f"{order['order_year']}-{order['order_month']}-1", axis=1)
order_per_month.head()

,order_month,order_year,count,date
0,1,2012,433,2012-1-1
4,2,2012,431,2012-2-1
8,3,2012,511,2012-3-1
12,4,2012,539,2012-4-1
16,5,2012,606,2012-5-1


In [140]:

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(order_per_month['date']), y=list(order_per_month['count'])))

fig.update_layout(
    title_text="Vendas por Mês e Ano"
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.update_traces(
    hovertemplate="Data: %{x} | Valor: %{y}",
    line={"width": 0.5},
    marker={"size": 8},
    mode="lines+markers",
    showlegend=False
)

fig.show()

### Vendas por continente e renda per capita


In [151]:
bestsellers = filter_columns(database, ['continent', 'country', 'gdpPercap'])

bestsellers = pd.DataFrame({
    'sales' : bestsellers.groupby(by=['continent', 'country', 'gdpPercap'])['continent'].count()
}).reset_index()

In [155]:
fig = px.scatter(
    bestsellers, 
    y="gdpPercap", 
    x="sales",
	size="sales", 
    color="continent",
    hover_name="country", 
    log_x=True, 
    size_max=60,
    title='Vendas por continente e renda per capita'
)
fig.show()

### Categorias de produto mais vendidos

In [180]:
bestsellers = filter_columns(database, ['continent', 'sub_category'])

bestsellers = pd.DataFrame({
    'sales' : bestsellers.groupby(by=['continent', 'sub_category'])['sub_category'].count()
}).reset_index()

bestsellers = bestsellers.sort_values(by="sales", ascending=False)
  
# bestsellers = bestsellers.groupby(by=['continent']).agg({ 'sub_category': pd.Series.mode }).reset_index()
bestsellers

,continent,sub_category,sales
20,Americas,Binders,2551
29,Americas,Paper,1956
53,Europe,Art,1878
54,Europe,Binders,1702
65,Europe,Storage,1697
...,...,...,...
67,Europe,Tables,119
69,Oceania,Appliances,107
79,Oceania,Machines,104
84,Oceania,Tables,53


In [184]:
import plotly.express as px

fig = px.bar(
    bestsellers, 
    x="sub_category", 
    y="sales", 
    color="continent", 
    title="Subcateria mais vendidas por continente",
    pattern_shape="continent"
)
fig.show()

### Mapa de calor das correlações

In [143]:
res = filter_columns(csv, ['sales', 'quantity', 'discount', 'profit', 'shipping_cost', 'delivery_days'])

In [144]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
					x=res.columns,
                    y=res.columns,
                    z=res.corr(),
                    text=res.corr(),
                    # text_auto=True,
                    texttemplate="%{text:.4f}",
                    textfont={"size": 16}))

fig.show()